### Import bibliotek

In [ ]:
from nltk.corpus import PlaintextCorpusReader

### Korpus dokumentów

In [ ]:
corpus_dir = "./Literatura - original"
corpus = PlaintextCorpusReader(corpus_dir, ".*\.txt")
files_names = corpus.fielids()
files_names